# 🎬 YTautoma - YouTube Shorts Automation

Generate 60-second YouTube Shorts using local AI models.

**Works on**: Colab (A100), RunPod, Lambda Labs

## 1️⃣ System Setup

In [ ]:
# Install system dependencies
!apt-get update -qq && apt-get install -y -qq ffmpeg
!ffmpeg -version | head -1

In [ ]:
# Upgrade PyTorch to 2.5+ (required for Z-Image)
!pip install -q --upgrade torch torchvision
import torch
print(f'PyTorch {torch.__version__}')
print(f'CUDA available: {torch.cuda.is_available()}')

## 2️⃣ Clone & Install

In [ ]:
import os

# Auto-detect workspace
if os.path.exists('/content'):
    WORKSPACE = '/content'
elif os.path.exists('/workspace'):
    WORKSPACE = '/workspace'
else:
    WORKSPACE = os.path.expanduser('~')

os.chdir(WORKSPACE)
print(f'Workspace: {WORKSPACE}')

# Clone repo
!git clone https://github.com/DragonLord1998/YTautoma.git 2>/dev/null || (cd YTautoma && git pull)
os.chdir('YTautoma')
PROJECT_DIR = os.getcwd()
print(f'Project: {PROJECT_DIR}')

In [ ]:
# Install Python dependencies
!pip install -q -r requirements.txt
!pip install -q git+https://github.com/huggingface/diffusers
!pip install -q edge-tts  # Simple TTS
!pip install -q flash-attn --no-build-isolation 2>/dev/null || echo 'flash-attn optional'

In [ ]:
# Install Ollama + Gemma 3
!curl -fsSL https://ollama.com/install.sh | sh

import subprocess, time
subprocess.Popen(['ollama', 'serve'], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
time.sleep(5)

!ollama pull gemma3:4b

In [ ]:
# Optional: Setup Wan 2.2 for video generation
!mkdir -p models
!git clone https://github.com/Wan-Video/Wan2.2.git models/Wan2.2 2>/dev/null || echo 'Already exists'
!pip install -q -r models/Wan2.2/requirements.txt

# Download TI2V-5B model (smaller, ~10GB)
# Uncomment to enable video generation:
# !pip install -q "huggingface_hub[cli]"
# !huggingface-cli download Wan-AI/Wan2.2-TI2V-5B --local-dir models/Wan2.2-TI2V-5B

In [ ]:
# Create .env configuration
import os
PROJECT_DIR = os.getcwd()

env_content = f"""OLLAMA_MODEL=gemma3:4b
OLLAMA_BASE_URL=http://localhost:11434

ZIMAGE_MODEL=Tongyi-MAI/Z-Image-Turbo
ZIMAGE_DEVICE=cuda

WAN_REPO_PATH={PROJECT_DIR}/models/Wan2.2
WAN_MODEL_PATH={PROJECT_DIR}/models/Wan2.2-TI2V-5B
WAN_T5_CPU=true
WAN_OFFLOAD_MODEL=true

TTS_ENGINE=edge

LOW_VRAM_MODE=true
TORCH_DTYPE=float16
"""

with open('.env', 'w') as f:
    f.write(env_content)

print('✅ Configuration saved!')

## 3️⃣ Generate YouTube Short

In [ ]:
# Quick test: Story only (no GPU needed)
!python main.py --story-only -c mystery

In [ ]:
# Generate story + images
!python main.py --images-only -c horror

In [ ]:
# Full pipeline (requires Wan 2.2 model downloaded)
# !python main.py -c sci-fi

## 4️⃣ View & Download

In [ ]:
# List outputs
!find output -name '*.png' -o -name '*.json' | head -30

In [ ]:
# View generated images
from IPython.display import Image, display
import glob

images = sorted(glob.glob('output/**/base_image.png', recursive=True))
for img in images[:6]:
    print(img)
    display(Image(filename=img, width=300))

In [ ]:
# Download (Colab)
import os, glob

try:
    from google.colab import files
    # Zip all outputs
    !cd output && zip -r ../output.zip .
    files.download('output.zip')
except ImportError:
    print('Not in Colab. Find outputs at: ./output/')